In [ ]:
# This assignment is taken from Svetlana Lazebnik.

### Common imports

In [ ]:
%matplotlib inline
import os
import sys
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import time
import random

### Provided functions
#### Image loading and saving

In [ ]:
def LoadFaceImages(pathname, subject_name, num_images):
    """
    Load the set of face images.  
    The routine returns
        ambimage: image illuminated under the ambient lighting
        imarray: a 3-D array of images, h x w x Nimages
        lightdirs: Nimages x 3 array of light source directions
    """

    def load_image(fname):
        return np.asarray(Image.open(fname))

    def fname_to_ang(fname):
        yale_name = os.path.basename(fname)
        return int(yale_name[12:16]), int(yale_name[17:20])

    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z

    ambimage = load_image(
        os.path.join(pathname, subject_name + '_P00_Ambient.pgm'))
    im_list = glob.glob(os.path.join(pathname, subject_name + '_P00A*.pgm'))
    if num_images <= len(im_list):
        im_sub_list = np.random.choice(im_list, num_images, replace=False)
    else:
        print(
            'Total available images is less than specified.\nProceeding with %d images.\n'
            % len(im_list))
        im_sub_list = im_list
    im_sub_list.sort()
#     improve_list = []
    
#     for fname in im_sub_list:
#         im_arr = load_image(fname)
#         count = 0
#         for x in range(im_arr.shape[0]):
#             for y in range(im_arr.shape[1]):
#                 if im_arr[x,y] < 40:
#                     count += 1
#         shadow = count / im_arr.size
#         if shadow < 0.4:
#             improve_list.append(fname)
#     im_sub_list = improve_list
    
    imarray = np.stack([load_image(fname) for fname in im_sub_list], axis=-1)
    Ang = np.array([fname_to_ang(fname) for fname in im_sub_list])

    x, y, z = sph2cart(Ang[:, 0] / 180.0 * np.pi, Ang[:, 1] / 180.0 * np.pi, 1)
    lightdirs = np.stack([y, z, x], axis=-1)
#     print(ambimage)
#     print(imarray)
#     print(ambimage.shape)
#     print(imarray.shape)
    return ambimage, imarray, lightdirs

In [ ]:
def save_outputs(subject_name, albedo_image, surface_normals):
    im = Image.fromarray((albedo_image*255).astype(np.uint8))
    im.save("%s_albedo.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,0]*128+128).astype(np.uint8))
    im.save("%s_normals_x.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,1]*128+128).astype(np.uint8))
    im.save("%s_normals_y.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,2]*128+128).astype(np.uint8))
    im.save("%s_normals_z.jpg" % subject_name)

#### Plot the albedo and the surface norms. 

In [ ]:
def plot_albedo_and_surface_normals(albedo_image, surface_normals):
    """
    albedo_image: h x w matrix
    surface_normals: h x w x 3 matrix.
    """
    fig, axes = plt.subplots(1, 4, figsize=(10,2.5))
    ax = axes[0]
    ax.axis('off')
    ax.set_title('albedo')
    ax.imshow(albedo_image, cmap='gray')

    ax = axes[1]
    ax.axis('off')
    ax.set_title('X')
    im = ax.imshow(surface_normals[:,:,0], cmap='jet')
    ax = axes[2]
    ax.axis('off')
    ax.set_title('Y')
    im = ax.imshow(surface_normals[:,:,1], cmap='jet')
    ax = axes[3]
    ax.axis('off')
    ax.set_title('Z')
    im = ax.imshow(surface_normals[:,:,2], cmap='jet')

    fig.colorbar(im, ax=axes, fraction=0.02, aspect=15)

#### Plot the height map

In [ ]:
def set_aspect_equal_3d(ax):
    """https://stackoverflow.com/questions/13685386"""
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([
        abs(lim - mean_)
        for lims, mean_ in ((xlim, xmean), (ylim, ymean), (zlim, zmean))
        for lim in lims
    ])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])

def display_3d(albedo_image, height_map, elev=20, azim=30):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.gca(projection='3d')
    ax.view_init(elev, azim)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

---
### Your implementation

In [ ]:
def preprocess(ambimage, imarray):
    """
    preprocess the data: 
        1. subtract ambient_image from each image in imarray.
        2. make sure no pixel is less than zero.
        3. rescale values in imarray to be between 0 and 1.
    Inputs:
        ambimage: h x w
        imarray: h x w x N images
    Outputs:
        processed_imarray: h x w x N images
    """
#     h,w,N = imarray.shape
#     imarray = imarray.reshape((N,h,w))
#     output = imarray - ambimage
    output = imarray - ambimage[:,:,None]
    output[output < 0] = 0
    processed_imarray = output/255
#     processed_imarray = processed_imarray.resize((h,w,N))
    return processed_imarray

In [ ]:
def photometric_stereo(imarray, light_dirs):
    """
    Inputs:
        imarray:  h x w x Nimages
        light_dirs: N images x 3
    Outputs:
        albedo_image: h x w
        surface_norms: h x w x 3   
    np.linalg.lstsq(a,b):
        return tuple with length 4 
        tuple[0]: solution; tuple[1]: residual; tuple[2]: rank of a; tuple[3]: sigular value of a
    """
    h,w,N = imarray.shape
    npix = h*w
#     imarray = imarray.reshape((N,npix))
    imarray = imarray.reshape((npix,N)).transpose()
    g, re, ra, sv = np.linalg.lstsq(light_dirs,imarray, None)
    albedo_image = np.linalg.norm(g, axis=0)    
    surface_normals = g / albedo_image  
    albedo_image = albedo_image.reshape((h,w))
    surface_normals = surface_normals.transpose().reshape((h,w,3))
    print(np.mean(re))
    return albedo_image, surface_normals

In [ ]:
def get_surface(surface_normals, integration_method):
    """
    Inputs:
        surface_normals:h x w x 3
        integration_method: string in ['average', 'column', 'row', 'random']
    Outputs:
        height_map: h x w
    """
    h,w,_ = surface_normals.shape
    fx = surface_normals[:,:,0]/surface_normals[:,:,2]
    fy = surface_normals[:,:,1]/surface_normals[:,:,2]
#     print(fx.shape)

    start = time.time()
    if integration_method == 'row':
        height_map = np.cumsum(fx, axis=1)[0] + np.cumsum(fy, axis=0)
    elif integration_method == 'column':
        sub_sum = np.cumsum(fy, axis=0)[:,0]
        height_map = np.cumsum(fx, axis =1) + sub_sum.reshape(len(sub_sum),1)
    elif integration_method == 'average':
        xsum = np.cumsum(fx, axis=1)
        ysum = np.cumsum(fy, axis=0)
        height_map = xsum[0] + ysum + xsum + ysum[:,0].reshape(len(ysum[:,0]),1)
        height_map /= 2
    elif integration_method == 'random':
        height_map = np.zeros((h,w))
        num_path = 30
        for i in range(num_path):
            for x in range(h):
                for y in range(w):
                    if x != 0 or y != 0:  
                        i, j, count, cumsum = 0, 0, 0, 0
                        # generate moving path 
                        path = np.hstack([np.zeros(x),np.ones(y)])
                        np.random.shuffle(path)

                        while i < x or j < y: 
                            if path[count] == 1:
                                cumsum += fx[i,j]
                                j += 1
                            else: 
                                cumsum += fy[i,j]
                                i += 1
                            count += 1
                        height_map[x,y] += cumsum
        
#         for i in range(num_path):
#             for x in range(h):
#                 for y in range(w):
#                     i, j, cumsum = 0, 0, 0
#                     while i <= x and j <= y:
#                         if i==x and j==y:
#                             break
#                         elif i == x:
#                             j += 1
#                             cumsum += fx[i,j] 
                            
#                         elif j == y:
#                             i += 1 
#                             cumsum += fy[i,j]
                            
#                         else:
#                             if random.randint(0,1):
#                                 j += 1
#                                 cumsum += fx[i,j]
                                
#                             else:
#                                 i += 1
#                                 cumsum += fy[i,j]
#                     height_map[x,y] += cumsum

        height_map /= num_path

                            
    end = time.time()
    print("Total execution time is: ", end-start, "s.")
    return height_map

### Main function

In [ ]:
root_path = './croppedyale/'
subject_name = 'yaleB01'
integration_method = 'average'
save_flag = True

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,
                                                    64)
processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray,
                                                   light_dirs)


# if save_flag:
#     save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_albedo_and_surface_normals(albedo_image, surface_normals)

In [ ]:
height_map = get_surface(surface_normals, 'row')
display_3d(albedo_image, height_map,20,40)
display_3d(albedo_image, height_map,20,-40)


In [ ]:
# display_3d(albedo_image, height_map,-10,-40)
# display_3d(albedo_image, height_map,-10,40)


In [ ]:
# height_map = get_surface(surface_normals, 'column')
# display_3d(albedo_image, height_map,20,40)
# display_3d(albedo_image, height_map,20,-40)



In [ ]:
# display_3d(albedo_image, height_map,-10,-40)
# display_3d(albedo_image, height_map,-10,40)


In [ ]:
# height_map = get_surface(surface_normals, 'average')
# display_3d(albedo_image, height_map,20,40)
# display_3d(albedo_image, height_map,20,-40)




In [ ]:
# display_3d(albedo_image, height_map,-10,-40)
# display_3d(albedo_image, height_map,-10,40)


In [ ]:
# height_map = get_surface(surface_normals, 'random')
# display_3d(albedo_image, height_map,20,30)
# display_3d(albedo_image, height_map,20,-30)



In [ ]:
# display_3d(albedo_image, height_map,-10,-40)
# display_3d(albedo_image, height_map,-10,40)
